
## Imports

The above imports are not at all mandatory in order to have the following code work. This cell is here to ensure that your setup is correct. 

In [5]:
%cd ..

%load_ext autoreload
%autoreload 2

/home/arthur/drl


In [8]:
#@title Imports  { form-width: "30%" }

%matplotlib inline
# import IPython
# from IPython.display import HTML
# from IPython import display as ipythondisplay

import acme
from acme import datasets
from acme import types
from acme import specs
from acme.wrappers import gym_wrapper
import base64
from base64 import b64encode
import chex
import collections
from collections import namedtuple
import dm_env
import enum
import functools
import gym
import haiku as hk
import imageio
import io
import itertools
import jax
from jax import tree_util
import optax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import multiprocessing as mp
import multiprocessing.connection
import numpy as np
import pandas as pd
import random
import reverb
import rlax
import time
import tree
from typing import *
import warnings
import pyglet
pyglet.options['search_local_libs'] = False
pyglet.options['shadow_window']=False
from pyglet.window import xlib
xlib._have_utf8 = False

from pyvirtualdisplay import Display
display = Display(visible=False, size=(1400, 900))
display.start()
 
np.set_printoptions(precision=3, suppress=1)
%matplotlib inline

## Random Agent on pendulum 


In [7]:
import acme

from src.environments.inverted_pendulum import InvertedPendulumEnv
from src.agents import RandomAgent
from src.interaction_loops import ddpg_parallel_interaction_loop
from src.plot_utils import display_video

/home/arthur/anaconda3/envs/drl/lib/python3.9/site-packages/matplotlib_inline/config.py:66: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


In [8]:
pendulum_environment = InvertedPendulumEnv(for_evaluation=True)  #### WARNING, I changed the env
pendulum_environment_spec = acme.make_environment_spec(pendulum_environment)
pendulum_random_agent = RandomAgent(pendulum_environment_spec)

ddpg_parallel_interaction_loop(pendulum_random_agent, pendulum_environment, 5, 10,10,10)

Process Process-1:
Traceback (most recent call last):
  File "/home/arthur/anaconda3/envs/drl/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/arthur/anaconda3/envs/drl/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/arthur/drl/mva-drl-project/src/utils.py", line 25, in actor_target
    environment = env_factory(seed=idx)
Process Process-2:
TypeError: 'InvertedPendulumEnv' object is not callable
Traceback (most recent call last):
  File "/home/arthur/anaconda3/envs/drl/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/arthur/anaconda3/envs/drl/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/arthur/drl/mva-drl-project/src/utils.py", line 25, in actor_target
    environment = env_factory(seed=idx)
TypeError: 'InvertedPendulumEnv' object is not callable


0


KeyboardInterrupt: 

In [10]:
display_video(np.stack(pendulum_environment.screens, axis=0))

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (500, 500) to (512, 512) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
[swscaler @ 0x7209780] Warning: data is not aligned! This can lead to a speed loss


## A2C agent

In [17]:
# import A2C specific
from src.agents import A2CAgent
from src.environments.inverted_pendulum import inverted_pendulum_env_factory
from src.interaction_loops import ddpg_parallel_interaction_loop, evaluation_parallel_interaction_loop
from src.plot_utils import plot_policy_on_pendulum

In [18]:
pendulum_environment = InvertedPendulumEnv(for_evaluation=True)  #### WARNING, I changed the env
pendulum_environment_spec = acme.make_environment_spec(pendulum_environment)
pendulum_random_agent = RandomAgent(pendulum_environment_spec)

In [21]:
# define the agent
num_evaluation_steps = 1000
max_learner_steps=50
sequence_length=4
# #pendulum_agent = RandomAgent(seed=0, 
# learning_rate=5e-4,
#  gamma=.95,
#   value_output_sizes=[256, 256],
#    policy_output_sizes=[256, 256], 
#    environment_spec=pendulum_environment_spec, 
#    entropy_loss_coef=3e-1)

In [24]:
  ddpg_parallel_interaction_loop(pendulum_random_agent, env_factory=inverted_pendulum_env_factory, max_learner_steps=max_learner_steps, sequence_length=sequence_length, num_actors=16, N=10, K=10)


0
10
here we are supposed to learn on a batch of size Transition(obs_tm1=DeviceArray([[-0.963329  ,  0.268323  , -0.87040573],
             [ 0.9854722 , -0.16983685, -0.7741058 ],
             [-0.9563103 ,  0.29235366,  0.657906  ],
             [-0.8633762 , -0.5045607 ,  0.10169578],
             [-0.61426747,  0.78909785,  0.22948302],
             [ 0.70591617,  0.7082954 , -0.6088262 ],
             [-0.9808427 ,  0.1948014 , -0.8773102 ],
             [ 0.6294431 ,  0.77704656, -0.43763173],
             [ 0.6936765 , -0.7202867 , -0.4451853 ]], dtype=float32), action_tm1=DeviceArray([[-0.23993424],
             [ 1.3539677 ],
             [ 1.1825701 ],
             [ 0.06954029],
             [ 0.33386442],
             [-1.3558552 ],
             [ 1.5485735 ],
             [ 0.13546419],
             [-0.69631016]], dtype=float32), reward_t=DeviceArray([-8.312379  , -0.09088345, -8.13817   , -6.8273425 ,
             -4.988334  , -0.65840113, -8.755558  , -0.8111993 ,
     

In [10]:
for _ in range(10):
  ddpg_parallel_interaction_loop(pendulum_random_agent, env_factory=inverted_pendulum_env_factory, max_learner_steps=max_learner_steps, sequence_length=sequence_length, num_actors=16)
  plot_policy_on_pendulum(pendulum_agent, 50)
  evaluation_parallel_interaction_loop(pendulum_agent, env_factory=inverted_pendulum_env_factory, sequence_length=200, num_actors=4)

NameError: name 'random_agent' is not defined